Testing connection and API keys

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

from openai import AsyncOpenAI
from anthropic import AsyncAnthropic
from exa_py import AsyncExa

Test OpenAI

In [ ]:
client = AsyncOpenAI()

response = await client.responses.create(
  model="gpt-4.1",
  input="Tell me a three sentence bedtime story about a unicorn."
)

print(response.output_text)

Under the silvery light of the moon, a gentle unicorn named Luna danced through a meadow sparkling with magical dew. She used her shimmering horn to paint rainbows across the night sky, filling the world with color and sweet dreams. As the stars twinkled above, Luna curled up in a soft patch of clover, knowing that tomorrow would bring even more adventures.


Test Anthropic

In [10]:
client = AsyncAnthropic()

message = await client.messages.create(
    max_tokens=1024,
    messages=[{
        "content": "Tell me a three sentence bedtime story about a unicorn.",
        "role": "user",
    }],
    model="claude-sonnet-4-5-20250929",
)

print(message.content[0].text)

Luna the unicorn galloped through moonlit meadows, her silver horn glowing softly in the night. She sprinkled stardust over sleeping flowers, helping them dream of spring. As dawn approached, she nestled beside a babbling brook and drifted into peaceful sleep, wrapped in morning mist.


Test EXA

In [4]:
exa = AsyncExa(api_key=os.environ.get("EXA_API_KEY"))

In [ ]:
result = await exa.search(
  "hottest AI startups",
  num_results=2
)

In [39]:
result.results[0].title

'Hottest AI Startups in the USA - LinkedIn'

In [40]:
result.results[0].url

'https://www.linkedin.com/pulse/hottest-ai-startups-usa-aitechsupports-ouwfc'

In [37]:
result.results[0].published_date

'2025-12-04T12:04:50.000Z'

In [43]:
print(result.results[0].text[:500])

Hottest AI Startups in the USA
Agree & Join LinkedIn
By clicking Continue to join or sign in, you agree to LinkedIn’s[User Agreement](https://www.linkedin.com/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement),[Privacy Policy](https://www.linkedin.com/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy), and[Cookie Policy](https://www.linkedin.com/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy).
``````````````
![]()## Sign in to view more content
Create y


In [48]:
answer_with_text = await exa.answer(
  query = "What are the hottest AI startups?",
  model = "exa-pro",
  text = True
)

In [51]:
answer_with_text.answer

'As of December 22, 2025, some of the hottest AI startups include Thinking Machines Lab, valued at $12 billion, and Glean, valued at $7.2 billion ([Reuters](https://www.reuters.com/technology/mira-muratis-ai-startup-thinking-machines-raises-2-billion-a16z-led-round-2025-07-15), [Reuters](https://www.reuters.com/technology/ai-company-glean-hits-72-billion-valuation-latest-funding-round-2025-06-10)). Other notable companies mentioned as "hottest" or "most promising" in 2025 include AI Squared, Morphos AI, and Writer ([CRN](https://www.crn.com/news/ai/2025/the-10-hottest-ai-startup-companies-of-2025-so-far)).'

EXA Search and Contents is deprecated --> search is the one to use

In [ ]:
query = "agentic memory for financial, private equity, due diligence use cases"

result = await exa.search(
    query=query,
    type="deep",
    moderation=False,
    num_results=100,
    contents={
        "text": {"maxCharacters": 100_000},
        "summary": True,
        #"highlights": True
    },
)

In [41]:
result.__dict__.keys()

dict_keys(['results', 'resolved_search_type', 'auto_date', 'context', 'statuses', 'cost_dollars'])

In [42]:
result.results[0].__dict__.keys()

dict_keys(['url', 'id', 'title', 'score', 'published_date', 'author', 'image', 'favicon', 'subpages', 'extras', 'text', 'summary', 'highlights', 'highlight_scores'])

In [48]:
result.results[0].url

'https://www.v7labs.com/agents/ai-private-equity-due-diligence-agent'

In [43]:
result.results[0].summary

"The webpage describes an **AI Private Equity Due Diligence Agent** designed to automate the first-pass diligence process, which directly addresses the user's interest in AI for financial and private equity due diligence use cases.\n\nHere is a summary of how this agent applies to those use cases:\n\n*   **Financial Due Diligence:** The agent automatically **extracts and normalizes historical financials**, calculates key metrics (like customer concentration and churn), and identifies commercial red flags. It can handle various financial documents like audited financials in scanned PDFs and Excel spreadsheets.\n*   **Private Equity Due Diligence (General Use Case):** The agent automates the entire process from the Virtual Data Room (VDR) analysis to generating a structured first draft of the **Investment Committee (IC) memo**. It reads, classifies, and indexes all VDR documents.\n*   **Risk Identification:** It performs a **Legal & Contractual Risk Review**, scanning contracts to flag n

In [44]:
result.results[0].highlights

['AI Private Equity Due Diligence Agent | Automate VDR Analysis | V7 Go Delegate your entire first-pass diligence to a specialized AI agent. It reads the whole data room, extracts and normalizes financials, identifies commercial and legal risks, and generates a structured first draft of your investment committee memo, letting your team focus on the final 10% of strategic analysis. See AI Private Equity Due Diligence Agent in action See AI Private Equity Due Diligence Agent in action']

In [46]:
result.results[0].highlight_scores

[0.337890625]

In [33]:
len(result.results[0].text)

18960

In [47]:
print(result.results[0].text)

AI Private Equity Due Diligence Agent | Automate VDR Analysis | V7 Go
[
](../)
[
Book a demo
](../demo)
[
](../)
[
Book a demo
](../demo)
[
](../agents)
Weeks of diligence in hours
# AI Private Equity Due Diligence Agent
## Go from data room to investment memo, faster
Delegate your entire first-pass diligence to a specialized AI agent. It reads the whole data room, extracts and normalizes financials, identifies commercial and legal risks, and generates a structured first draft of your investment committee memo, letting your team focus on the final 10% of strategic analysis.
[
Book a demo
](../demo)
![](https://framerusercontent.com/images/fiul9FtpI0Kq8BYEzpNW2KjKYc.png?width=662&amp;height=576)
Ideal for
Private Equity
Growth Equity
Venture Capital
![](https://framerusercontent.com/images/Pf3dS4BD0DF6sqndarcpaDxQc.jpg?width=1111&amp;height=1111)
* ![Mercedes-Benz logo](https://framerusercontent.com/images/yhdelSpUYTozFf9zSxbV6t5gFao.svg?width=180&amp;height=72)
![SMC logo](https://fram

EXA WebSets

https://docs.exa.ai/websets/api/get-started

In [60]:
from exa_py.websets.types import CreateWebsetParameters, CreateEnrichmentParameters
from exa_py import Exa

exa = Exa(os.getenv('EXA_API_KEY'))

In [77]:
preview = exa.websets.preview(params={
    'query': 'Latest AI research papers that are applicable to private equity'
})

In [78]:
preview.search.entity

WebsetResearchPaperEntity(type='research_paper')

In [79]:
for e, i in enumerate(preview.search.criteria):
    print(f"{e+1}: {i.description}")
    print("===" * 30)

1: Research paper focused on artificial intelligence (AI) applications or relevance to private equity
2: Published during 2024 or 2025


arXiv

In [3]:
import sys
sys.path.append("..")
from components.tools.arxiv import search_arxiv, print_results, get_arxiv_categories

In [4]:
arxiv_categories = get_arxiv_categories()
print(arxiv_categories)

['cs.AI', 'cs.CE', 'cs.CL', 'cs.DB', 'cs.DC', 'cs.GL', 'cs.GT', 'cs.HC', 'cs.IR', 'cs.LG', 'cs.MA', 'cs.NE', 'econ.GN', 'math.ST', 'q-fin.CP', 'q-fin.GN', 'q-fin.MF', 'q-fin.PM', 'q-fin.RM', 'q-fin.ST', 'q-fin.TR', 'stat.ML']


In [5]:
# =============================================================================
# Example: Search across multiple categories (AI, NLP, Finance)
# =============================================================================
feed = search_arxiv(
    categories=arxiv_categories,
    max_results=5,
    sort_by="submittedDate",
    sort_order="descending"
)
print_results(feed)

Query URL: http://export.arxiv.org/api/query?search_query=%28cat%3Acs.AI+OR+cat%3Acs.CE+OR+cat%3Acs.CL+OR+cat%3Acs.DB+OR+cat%3Acs.DC+OR+cat%3Acs.GL+OR+cat%3Acs.GT+OR+cat%3Acs.HC+OR+cat%3Acs.IR+OR+cat%3Acs.LG+OR+cat%3Acs.MA+OR+cat%3Acs.NE+OR+cat%3Aecon.GN+OR+cat%3Amath.ST+OR+cat%3Aq-fin.CP+OR+cat%3Aq-fin.GN+OR+cat%3Aq-fin.MF+OR+cat%3Aq-fin.PM+OR+cat%3Aq-fin.RM+OR+cat%3Aq-fin.ST+OR+cat%3Aq-fin.TR+OR+cat%3Astat.ML%29&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending

Total results: 516269
Showing: 5 entries


[1] LongVideoAgent: Multi-Agent Reasoning with Long Videos
    ID: http://arxiv.org/abs/2512.20618v1
    Published: 2025-12-23T18:59:49Z
    Authors: Runtao Liu, Ziyi Liu, Jiaqi Tang + 4 more
    Categories: cs.AI, cs.CV, cs.LG, cs.MA
    Summary: Recent advances in multimodal LLMs and systems that use tools for long-video QA point to the promise of reasoning over hour-long episodes. However, many methods still compress content into lossy summa...
----------------------

In [8]:
# =============================================================================
# Example: Keyword search
# =============================================================================
feed = search_arxiv(
    search_query="\"private equity\" AND \"agent\"", #"LLM OR \"large language model\""
    #categories=arxiv_categories,
    max_results=5,
    sort_by="submittedDate",
    sort_order="descending"
)
print_results(feed)

Query URL: http://export.arxiv.org/api/query?search_query=all%3A%22private+equity%22+AND+%22agent%22&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending

Total results: 2
Showing: 2 entries


[1] Finance Agent Benchmark: Benchmarking LLMs on Real-world Financial Research Tasks
    ID: http://arxiv.org/abs/2508.00828v1
    Published: 2025-05-20T18:22:10Z
    Authors: Antoine Bigeard, Langston Nashold, Rayan Krishnan + 1 more
    Categories: cs.CE
    Summary: Artificial Intelligence (AI) technology has emerged as a transformative force in financial analysis and the finance industry, though significant questions remain about the full capabilities of Large L...
--------------------------------------------------------------------------------

[2] Solving the Equity Risk Premium Puzzle and Inching Towards a Theory of Everything
    ID: http://arxiv.org/abs/1604.04872v7
    Published: 2016-04-17T14:07:15Z
    Authors: Ravi Kashyap
    Categories: q-fin.GN, econ.GN
    Summary: Th